# English Data PreProcessing

## [주요 고려 사항]
1. `dot(.)`과 `apostrophe(')` 처리
    - 'u.s.'와 'u.s.s.r.'과 같은 약자처리를 어떻게 할 것인가?
    - 'america's'와 같은 소유격을 어떻게 처리할 것인가?
        1. 처음 Cleaning 때, `dot(.)`과 `apostrophe(')`는 제거하지 않음
            - `dot(.)`
                - 'u.s', 'u.s.s.r'과 같은 약자를 유지시키기 위한 처리
            - `apostrophe(')`
                - 'america's'와 같은 소유격을 유지시켜서 Tokenizing때 's를 분리시키기 위함.
        2. Tokenizing 이후, `dot(.)`과 `apostrophe(')`를 유지시켜야 하는 Token들 외에는 특수문자 제거
            1. `apostrophe(')`와 `dot(.)`을 가진 Token들을 출력해보고 유지시킬 Token들의 목록을 결정
            2. `apostrophe(')`를 유지시킬 Token들 외의 모든 Token들에서 `apostrophe(')` 및 특수문자 제거
                - `dot(.)`은 다음 단계에서 예외처리를 하며 제거해야 하므로, 이 단계에서는 모든 `dot(.)`을 유지시킴
            3. `dot(.)`을 유지시킬 Token들 외의 모든 Token들에서 `dot(.)` 및 특수문자 제거

## Module Import

In [1]:
# self defined Modules
from myModules.utils.data.DataLoader import DataLoader
from myModules.utils.merge.mergeOverPeriod import merge
from myModules.preprocess import cleaning, removeStopWords_ST, tagging, extract_some_pos_ST

# General Modules
import pandas as pd
import numpy as np
import warnings
from tqdm.notebook import tqdm
import pickle
import re
import glob

warnings.filterwarnings('ignore')

# Read File
import glob

# NLP
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Data Loader

In [2]:
DATA_ROOT = './Data/3구간/'

PERIOD_1 = DATA_ROOT + '1시기/1시기_ST/'
PERIOD_2 = DATA_ROOT + '2시기/2시기_ST/'
PERIOD_3 = DATA_ROOT + '3시기/3시기_ST/'

RESULT_ROOT = './Result/3구간/'

RESULT_1 = RESULT_ROOT + '/1시기/ST/'
RESULT_2 = RESULT_ROOT + '/2시기/ST/'
RESULT_3 = RESULT_ROOT + '/3시기/ST/'

In [3]:
files_1 = glob.glob(PERIOD_1+'*.txt')
files_2 = glob.glob(PERIOD_2+'*.txt')
files_3 = glob.glob(PERIOD_3+'*.txt')

texts_1 = DataLoader(files_1, mode='ST')
texts_2 = DataLoader(files_2, mode='ST')
texts_3 = DataLoader(files_3, mode='ST')

## PreProcess

### 3-1. Data Cleaning

- `dot(.)`과 `apostrophe(')`는 제거하지 않음

In [4]:
cleaned_1 = cleaning(texts=texts_1, mode='ST')
cleaned_2 = cleaning(texts=texts_2, mode='ST')
cleaned_3 = cleaning(texts=texts_3, mode='ST')

### 3-2. Tokenizing

In [5]:
class dot_and_apostrophe:
    def __init__(self, data):
        self.data = data
    
    def token_with_apostrophe(self):
        apostrophe = []

        for tokens in self.data:
            for token in tokens:
                if "'" in token : apostrophe.append(token)
        
        self.apostrophes = set(apostrophe)

        print(f"apostrophe를 가진 token : \n{self.apostrophes}")
    
    def token_with_dot(self):
        dot = []

        for tokens in self.data:
            for token in tokens:
                if "." in token : dot.append(token)
        
        self.dots = set(dot)

        print(f"dot을 가진 token : \n{self.dots}")
        
    def set_exception(self, apostrophe_exception, dot_exception):
        self.apostrophe_exception = apostrophe_exception
        self.dot_exception = dot_exception
    
    def print_exception(self):
        print(f"apostrophe exceptions : \n{self.apostrophe_exception}")
        print(f"dot exceptions : \n{self.dot_exception}")
    
    def remove_apostrophe(self, data):
        result = []
        processed = []

        for tokens in data:
            arr = []
            for token in tokens:
                if token not in self.apostrophe_exception:
                    if not token.isalnum() : 
                        if "." not in token : processed.append(token)
                    # dot은 삭제하지 않음. -> 예외처리하면서 삭제해야함
                    arr.append(re.sub("[^a-z\.]", "", token))
                else : arr.append(token)
            result.append(arr)
        
        processed = set(processed)

        print(f"Processed Tokens : \n{processed}")
        
        return result
    
    def remove_dot(self, data):
        result = []
        processed = []

        for tokens in data:
            arr = []
            for token in tokens:
                if token not in self.dot_exception:
                    if not token.isalnum() : 
                        if "'" not in token : processed.append(token)
                    # apostrophe는 삭제하지 않음. -> 예외처리하면서 삭제
                    arr.append(re.sub("[^a-z']", "", token))
                else : arr.append(token)
            result.append(arr)
        
        processed = set(processed)

        print(f"Processed Tokens : \n{processed}")
        
        return result
    
    def check_invalid_tokens(self, data):
        # 예외처리한 Token들 외에 특수문자를 가진 Token들이 있는지 확인
        invalid_tokens = []

        for tokens in data:
            for token in tokens:
                if not token.isalnum() : invalid_tokens.append(token)
                elif len(token) == 1 : invalid_tokens.append(token)
        
        invalid_tokens = set(invalid_tokens)
        exception = set(self.apostrophe_exception).union(set(self.dot_exception))
        self.invalid_symbol = invalid_tokens.difference(exception)

        if len(self.invalid_symbol) == 0:
            print("There is no invalid symbol")
        else :
            print(f"Remaining invalid Symbol : {self.invalid_symbol}")
    
    def remove_invalid_tokens(self, data):
        # 남아있는 특수문자 + 길이가 1인 token들을 삭제
        
        result = []
        removed = []

        for tokens in data:
            arr = []
            for token in tokens:
                if len(token) == 1 : removed.append(token)
                elif token in self.invalid_symbol : removed.append(token)
                else : arr.append(token)
            result.append(arr)

        removed = set(removed)
        
        print(f"Removed Tokens : \n{removed}")

        return result

In [6]:
tokenized_1 = [word_tokenize(text) for text in cleaned_1]
tokenized_2 = [word_tokenize(text) for text in cleaned_2]
tokenized_3 = [word_tokenize(text) for text in cleaned_3]

#### Period 1

In [7]:
symbol = dot_and_apostrophe(tokenized_1)

##### apostrophe와 dot을 가진 token들 시각화

In [8]:
symbol.token_with_apostrophe()
symbol.token_with_dot()

apostrophe를 가진 token : 
{"'ve", "'ll", "'into", "o'clock", "'m", "'system", "'s", "'mvd", "'liberty", "'", "'german", "'are", "'d", "'blamed", "'madam", "'structure", "'democracy", "n't", "'heat"}
dot을 가진 token : 
{'u.s.', 'gen.', 'a.m.', 'p.', 'n.', 'a.', 'mr.', 'jr.', 'u.s.s.r.', 'mrs.', 'm.', 'frightened.to', 'v.', 'u.n.', '...', 'st.', 'oct.', 'col.', 'messrs.', 'camps.if', 'i.', 's.', 'p.m.', 'ph.d.', '..', 'u.', 'e.', 'dr.', 'co.', 't.', '.', 'f.', 'w.'}


##### exception 목록 설정

In [9]:
apostrophe_exception = ["'ll", "'s", "'ve"]
dot_exception = ["u.s.s.r.", "dr.", "messrs.", "gen.", "u.n.", "a.m.", "st.", "u.s.", "ph.d", "jr.", "p.m.", "mrs.", "mr."]

symbol.set_exception(apostrophe_exception=apostrophe_exception, dot_exception=dot_exception)

In [10]:
symbol.print_exception()

apostrophe exceptions : 
["'ll", "'s", "'ve"]
dot exceptions : 
['u.s.s.r.', 'dr.', 'messrs.', 'gen.', 'u.n.', 'a.m.', 'st.', 'u.s.', 'ph.d', 'jr.', 'p.m.', 'mrs.', 'mr.']


##### apostrophe 처리

In [11]:
tokenized_1_ = symbol.remove_apostrophe(data=tokenized_1)

Processed Tokens : 
{"'mvd", "'madam", "'are", "'structure", "'democracy", "'into", "n't", "'liberty", "'", "'m", "o'clock", "'d", "'heat", "'system", "'german", "'blamed"}


##### dot 처리

In [12]:
tokenized_1__ = symbol.remove_dot(data=tokenized_1_)

Processed Tokens : 
{'', 'p.', 'n.', 'a.', 'm.', 'frightened.to', 'v.', '...', 'oct.', 'col.', 'camps.if', 'i.', 's.', 'ph.d.', '..', 'u.', 'e.', 'co.', 't.', '.', 'f.', 'w.'}


##### 제거해야할 token 검사

In [13]:
symbol.check_invalid_tokens(data=tokenized_1__)

Remaining invalid Symbol : {'', 'v', 'i', 'x', 'r', 'd', 'g', 'o', 'y', 'k', 's', 'b', 'u', 'w', 'h', 'e', 'm', 't', 'n', 'a', 'f', 'j', 'p'}


##### 길이가 1이거나 필요없는 특수문자인 Token들 삭제

In [14]:
tokenized_1___ = symbol.remove_invalid_tokens(data=tokenized_1__)

Removed Tokens : 
{'', 'v', 'i', 'x', 'r', 'd', 'g', 'o', 'y', 'k', 's', 'b', 'u', 'w', 'h', 'e', 'm', 't', 'n', 'a', 'f', 'j', 'p'}


##### 남아있는 invalid한 token이 있는지 검사

In [15]:
symbol.check_invalid_tokens(data=tokenized_1___)

There is no invalid symbol


#### Peiod 2

In [16]:
symbol = dot_and_apostrophe(tokenized_2)

##### apostrophe와 dot을 가진 token들 시각화

In [17]:
symbol.token_with_apostrophe()
symbol.token_with_dot()

apostrophe를 가진 token : 
{"'for", "n't", "'", "'m", "'reprisals", "'s"}
dot을 가진 token : 
{'gen.', 'p.', 'n.', 'a.', 'b.', 'mr.', 'u.s.s.r.', 'm.', 'v.', 'g.', 'r.', 'messrs.', 'i.', 's.', 'o.', 'l.', 'p.m.', '..', 'u.s.a.', 'c.', 'h.', 'e.', 'dr.', 't.', '.', 'tyranny.the', 'w.'}


##### exception 목록 설정

In [18]:
apostrophe_exception = ["'s"]
dot_exception = ["u.s.s.r.", "dr.", "messrs.", "gen.", "u.s.a.", "p.m.", "mr."]

symbol.set_exception(apostrophe_exception=apostrophe_exception, dot_exception=dot_exception)

In [19]:
symbol.print_exception()

apostrophe exceptions : 
["'s"]
dot exceptions : 
['u.s.s.r.', 'dr.', 'messrs.', 'gen.', 'u.s.a.', 'p.m.', 'mr.']


##### apostrophe 처리

In [20]:
tokenized_2_ = symbol.remove_apostrophe(data=tokenized_2)

Processed Tokens : 
{"'for", "n't", "'", "'m", "'reprisals"}


##### dot 처리

In [21]:
tokenized_2__ = symbol.remove_dot(data=tokenized_2_)

Processed Tokens : 
{'', 'p.', 'n.', 'a.', 'b.', 'm.', 'v.', 'g.', 'r.', 'i.', 's.', 'o.', 'l.', '..', 'c.', 'h.', 'e.', 't.', '.', 'tyranny.the', 'w.'}


##### 제거해야할 Token들 검사

In [22]:
symbol.check_invalid_tokens(data=tokenized_2__)

Remaining invalid Symbol : {'', 'v', 'i', 'l', 'r', 'd', 'g', 'o', 's', 'b', 'w', 'h', 'c', 'e', 'm', 't', 'n', 'a', 'f', 'p'}


##### 길이가 1이거나 필요없는 특수문자인 token 제거

In [23]:
tokenized_2___ = symbol.remove_invalid_tokens(data=tokenized_2__)

Removed Tokens : 
{'', 'i', 'v', 'l', 'r', 'd', 'g', 'o', 's', 'b', 'w', 'h', 'c', 'e', 'm', 't', 'n', 'a', 'f', 'p'}


##### 남아있는 Invalid한 Token이 있는지 확인

In [24]:
symbol.check_invalid_tokens(data=tokenized_2___)

There is no invalid symbol


#### period 3

In [25]:
symbol = dot_and_apostrophe(tokenized_3)

##### apostrophe와 dot을 가진 token들 시각화

In [26]:
symbol.token_with_apostrophe()
symbol.token_with_dot()

apostrophe를 가진 token : 
{"'vas", "'ve", "'ll", "'has", "'spontaneous", "n't", "'", "o'clock", "'d", "'recession", "'s"}
dot을 가진 token : 
{'gen.', 'a.m.', 'p.', 'n.', 'j.', 'a.', 'b.', 'mr.', 'u.s.s.r.', 'mrs.', 'jr.', 'm.', 'v.', 'g.', '...', 'd.', '..................', 'st.', 'prof.', 'r.', 'u.n.r.r.a', 'col.', 'i.', 's.s.r', 's.', 'o.', 'l.', 'p.m.', 'u.', 'c.', 'h.', 'e.', 'dr.', 'maj.', 't.', '.', 'f.', 'w.'}


##### exception 목록 설정

In [27]:
apostrophe_exception = ["'ll", "'s", "'ve"]
dot_exception = ["u.s.s.r.", "dr.", "s.s.r", "a.m.", "st.", "prof.", "u.n.r.r.a", "jr.", "maj.", "p.m.", "mrs.", "mr."]

symbol.set_exception(apostrophe_exception=apostrophe_exception, dot_exception=dot_exception)

In [28]:
symbol.print_exception()

apostrophe exceptions : 
["'ll", "'s", "'ve"]
dot exceptions : 
['u.s.s.r.', 'dr.', 's.s.r', 'a.m.', 'st.', 'prof.', 'u.n.r.r.a', 'jr.', 'maj.', 'p.m.', 'mrs.', 'mr.']


##### apostrophe 처리

In [29]:
tokenized_3_ = symbol.remove_apostrophe(tokenized_3)

Processed Tokens : 
{"'vas", '``', "'has", "'spontaneous", "n't", "'", "o'clock", "'d", "'recession"}


##### dot 처리

In [30]:
tokenized_3__ = symbol.remove_dot(tokenized_3_)

Processed Tokens : 
{'', 'gen.', 'p.', 'n.', 'j.', 'a.', 'b.', 'm.', 'v.', 'g.', '...', 'd.', '..................', 'r.', 'col.', 'i.', 's.', 'o.', 'l.', 'u.', 'c.', 'h.', 'e.', 't.', '.', 'f.', 'w.'}


##### 제거해야할 token 확인

In [31]:
symbol.check_invalid_tokens(tokenized_3__)

Remaining invalid Symbol : {'', 'v', 'i', 'x', 'l', 'r', 'd', 'g', 'o', 's', 'b', 'u', 'w', 'h', 'c', 'e', 'm', 't', 'n', 'a', 'f', 'j', 'p'}


##### 길이가 1이거나 필요없는 특수문자인 token 제거

In [32]:
tokenized_3___ = symbol.remove_invalid_tokens(tokenized_3__)

Removed Tokens : 
{'', 'i', 'v', 'x', 'l', 'r', 'd', 'g', 'o', 's', 'b', 'u', 'w', 'h', 'c', 'e', 'm', 't', 'n', 'a', 'f', 'j', 'p'}


##### 남아있는 INvalid한 token이 있는지 확인

In [33]:
symbol.check_invalid_tokens(tokenized_3___)

There is no invalid symbol


### 3-3. Remove StopWords

In [34]:
stopwords = nltk.corpus.stopwords.words('english')
new_stopwords = ['would', 'could', 'might', 'need', 'can', 'must', \
    'one', 'two', 'upon', 'may', 'perhaps', 'living', 'seem', 'also', 'ii', 'ofthe',
    'also', 'much', 'therefore']

wo_stopword_1 = removeStopWords_ST(tokenized_1___, stopwords, new_stopwords)
wo_stopword_2 = removeStopWords_ST(tokenized_2___, stopwords, new_stopwords)
wo_stopword_3 = removeStopWords_ST(tokenized_3___, stopwords, new_stopwords)

### 3-4. Lemmatization

In [51]:
def lemmatizing(data, lemmatizer):
    result = []

    for article in data:
        arr = []
        for token in article:
            if "." in token : arr.append(token)
            elif "'" in token : arr.append(token)
            elif token == "us" : arr.append(token)
            else : 
                lemmatized = lemmatizer.lemmatize(token)
                # if len(lemmatized) == 1: print(f"token : {token} | lemmatized : {lemmatized}")
                arr.append(lemmatized)
        result.append(arr)
    
    return result

In [52]:
lemmatizer = WordNetLemmatizer()

lemmatized_1 = lemmatizing(wo_stopword_1, lemmatizer)
lemmatized_2 = lemmatizing(wo_stopword_2, lemmatizer)
lemmatized_3 = lemmatizing(wo_stopword_3, lemmatizer)

### 3-5. Tagging

In [54]:
tagList = [['noun', ['NN','NNS','NNP','NNPS'], ['NNG','NNB','NNP','NNM']], \
    ['pronoun', ['PRP','WP','PRP'], ['NP']],
    ['verb', ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'], ['VV', 'VXV', 'VCP']],
    ['adjective', ['JJ', 'JJR', 'JJS'], ['VA', 'VXA', 'VCN']],
    ['adverb', ['RB', 'RBR', 'RBS', 'WRB', 'EX', 'RP'], ['MAG']],
    ['prep&conj', ['TO', 'IN', 'CC'], ['MAC']],
    ['determiner', ['DT', 'PDT', 'WDT'], ['MDT', 'MDN']],
    ['interjection',['UH'], ['IC']],
    ['number', ['CD'], ['NR', 'ON']],
    ['foreignW', ['FW'],['OL']],
    ['modal',['MD'],[]],
    ['josa', [], ['JC', 'JK', 'JKC', 'JKG', 'JKI', 'JKM', 'JKO', 'JKQ', 'JKS', 'JX']],
    ['possesiveS', ['POS'], []],
    ['others',['LS'], ['EPH', 'EPT', 'EPP', 'EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR', 'ECE', 'ECD', 'ECS', 'ETN', 'ETD', 'XPN', 'XPV', 'XSN', 'XSV', 'XSA', 'XR', 'UN', 'OH']]]

tagList = pd.DataFrame(tagList)
tagList.columns = ['POS', 'Eng_tag', 'Kor_tag']

In [55]:
tagged_1 = tagging(lemmatized_1, mode='ST')
tagged_2 = tagging(lemmatized_2, mode='ST')
tagged_3 = tagging(lemmatized_3, mode='ST')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

##### POS 분류 결과 검증

[검증 내용]
1. dot을 포함한 token이 올바르게 분류되었는가?
2. apostropphe를 포함한 token이 올바르게 분류되었는가?

In [56]:
class verify_tag:
    def __init__(self, data):
        self.data = data
    
    def token_with_symbol(self):
        dots = []
        apostrophes = []

        for tags in self.data:
            for tag in tags:
                if "." in tag[0] : dots.append(tag)
                elif "'" in tag[0] : apostrophes.append(tag)
        
        self.apostrophes = set(apostrophes)
        self.dots = set(dots)

        print(f"tagged token include apostrophe : \n{set(apostrophes)}")
        print(f"tagged token include dot : \n{set(dots)}") 

    def verify_dot_token(self):
        for tag in self.dots:
            removed = nltk.pos_tag([re.sub("[^a-z]", "", tag[0])])
            print(f"{tag[0]} -> {tag[1]}   |   {removed[0][0]} -> {removed[0][1]}")
    
    def verify_apostrophe_token(self):
        for tag in self.apostrophes:
            removed = nltk.pos_tag([re.sub("[^a-z]", "", tag[0])])
            print(f"{tag[0]} -> {tag[1]}   |   {removed[0][0]} -> {removed[0][1]}")

#### Period 1

In [57]:
verify = verify_tag(data=tagged_1)

##### symbol을 가진 token들을 출력

In [58]:
verify.token_with_symbol()

tagged token include apostrophe : 
{("'ve", 'VBP'), ("'s", 'POS'), ("'ll", 'MD')}
tagged token include dot : 
{('mr.', 'VBZ'), ('u.n.', 'NN'), ('a.m.', 'JJ'), ('gen.', 'NN'), ('mrs.', 'NNS'), ('u.s.', 'JJ'), ('st.', 'NN'), ('mr.', 'JJ'), ('jr.', 'VBP'), ('gen.', 'JJ'), ('messrs.', 'NN'), ('jr.', 'NN'), ('dr.', 'NN'), ('mr.', 'NNP'), ('st.', 'JJ'), ('mr.', 'VBP'), ('u.s.s.r.', 'JJ'), ('dr.', 'VB'), ('p.m.', 'RB'), ('mr.', 'NN')}


##### dot을 가진 token 검증

- dot을 넣고 빼서 Pos tagging을 수행하고 각각의 결과가 올바른지 결정.
- gen. -> Gulag is headed by Gen. Victor nedosyekin, first vice minister of internal affairs, 에서 나옴.
    - 사람 이름인 듯 하다.
- **messrs., mr., dr., gen., jr., st., mrs., a.m., p.m.**
    - '약어'로 tagging이 되어야 할 듯 하다.
- **u.s.s.r., u.s., u.n.**
    - '명사'로 tagging이 되어야 할 듯 하다.
- 수정 방안
    - 각각의 올바른 tag로 변경 해주기
    - 확실한 구분을 위해 dot(.)은 유지시킨 채로 각 token들의 tag 바꾸어 주는 방향으로 전처리

In [59]:
verify.verify_dot_token()

mr. -> VBZ   |   mr -> NN
u.n. -> NN   |   un -> NN
a.m. -> JJ   |   am -> VBP
gen. -> NN   |   gen -> NN
mrs. -> NNS   |   mrs -> NN
u.s. -> JJ   |   us -> PRP
st. -> NN   |   st -> NN
mr. -> JJ   |   mr -> NN
jr. -> VBP   |   jr -> NN
gen. -> JJ   |   gen -> NN
messrs. -> NN   |   messrs -> NN
jr. -> NN   |   jr -> NN
dr. -> NN   |   dr -> NN
mr. -> NNP   |   mr -> NN
st. -> JJ   |   st -> NN
mr. -> VBP   |   mr -> NN
u.s.s.r. -> JJ   |   ussr -> NN
dr. -> VB   |   dr -> NN
p.m. -> RB   |   pm -> NN
mr. -> NN   |   mr -> NN


##### apostrophe를 가진 token 처리

- apostrophe(')를 포함한 채로 tagging 하는 것이 더 정확하게 동작하는 것을 확인하였다.

In [60]:
verify.verify_apostrophe_token()

've -> VBP   |   ve -> NN
's -> POS   |   s -> NN
'll -> MD   |   ll -> NN


#### Period 2

In [61]:
verify = verify_tag(data=tagged_2)

##### symbol을 가진 token들 출력

In [62]:
verify.token_with_symbol()

tagged token include apostrophe : 
{("'s", 'POS')}
tagged token include dot : 
{('mr.', 'NNP'), ('gen.', 'JJ'), ('mr.', 'VBZ'), ('messrs.', 'NN'), ('mr.', 'FW'), ('p.m.', 'JJ'), ('dr.', 'NN'), ('mr.', 'VBP'), ('mr.', 'JJ'), ('u.s.s.r.', 'JJ'), ('mr.', 'VBD'), ('mr.', 'NNS'), ('mr.', 'RBS'), ('mr.', 'VB'), ('mr.', 'NN'), ('messrs.', 'NNS'), ('u.s.a.', 'NN')}


##### dot을 가진 token 검증

- dot을 넣고 빼서 Pos tagging을 수행하고 각각의 결과가 올바른지 결정.
- gen. -> Gulag is headed by Gen. Victor nedosyekin, first vice minister of internal affairs, 에서 나옴.
    - 사람 이름인 듯 하다.
- **messrs., mr., dr., gen., jr., st., mrs., a.m., p.m.**
    - '약어'로 tagging이 되어야 할 듯 하다.
- **u.s.s.r., u.s.a., u.n.**
    - '명사'로 tagging이 되어야 할 듯 하다.
- 수정 방안
    - 각각의 올바른 tag로 변경 해주기
    - 확실한 구분을 위해 dot(.)은 유지시킨 채로 각 token들의 tag 바꾸어 주는 방향으로 전처리

In [63]:
verify.verify_dot_token()

mr. -> NNP   |   mr -> NN
gen. -> JJ   |   gen -> NN
mr. -> VBZ   |   mr -> NN
messrs. -> NN   |   messrs -> NN
mr. -> FW   |   mr -> NN
p.m. -> JJ   |   pm -> NN
dr. -> NN   |   dr -> NN
mr. -> VBP   |   mr -> NN
mr. -> JJ   |   mr -> NN
u.s.s.r. -> JJ   |   ussr -> NN
mr. -> VBD   |   mr -> NN
mr. -> NNS   |   mr -> NN
mr. -> RBS   |   mr -> NN
mr. -> VB   |   mr -> NN
mr. -> NN   |   mr -> NN
messrs. -> NNS   |   messrs -> NN
u.s.a. -> NN   |   usa -> NN


##### apostrophe를 가진 token 처리

- apostrophe(')를 포함한 채로 tagging 하는 것이 더 정확하게 동작하는 것을 확인하였다.

In [64]:
verify.verify_apostrophe_token()

's -> POS   |   s -> NN


#### Period 3

In [65]:
verify = verify_tag(data=tagged_3)

##### symbol을 가진 token들을 출력

In [66]:
verify.token_with_symbol()

tagged token include apostrophe : 
{("'ve", 'VBP'), ("'s", 'POS'), ("'ll", 'MD')}
tagged token include dot : 
{('mr.', 'VBZ'), ('mr.', 'FW'), ('dr.', 'JJ'), ('a.m.', 'VBD'), ('mr.', 'VB'), ('a.m.', 'RB'), ('u.n.r.r.a', 'JJ'), ('mrs.', 'NN'), ('p.m.', 'NN'), ('prof.', 'NN'), ('u.n.r.r.a', 'RB'), ('mr.', 'JJ'), ('mr.', 'VBD'), ('s.s.r', 'NN'), ('jr.', 'NN'), ('dr.', 'NN'), ('a.m.', 'NN'), ('dr.', 'VBZ'), ('mr.', 'NNP'), ('st.', 'JJ'), ('mr.', 'RBR'), ('maj.', 'NN'), ('u.s.s.r.', 'JJ'), ('mr.', 'NNS'), ('p.m.', 'RB'), ('mr.', 'NN')}


##### dot을 가진 token 검증

- dot을 넣고 빼서 Pos tagging을 수행하고 각각의 결과가 올바른지 결정.
- gen. -> Gulag is headed by Gen. Victor nedosyekin, first vice minister of internal affairs, 에서 나옴.
    - 사람 이름인 듯 하다.
- **messrs., mr., dr., gen., maj., prof.,  jr., st., mrs., a.m., p.m.**
    - '약어'로 tagging이 되어야 할 듯 하다.
- **u.s.s.r., u.s.a., u.n., s.s.r, u.n.r.r.a**
    - '명사'로 tagging이 되어야 할 듯 하다.
- 수정 방안
    - 각각의 올바른 tag로 변경 해주기
    - 확실한 구분을 위해 dot(.)은 유지시킨 채로 각 token들의 tag 바꾸어 주는 방향으로 전처리

In [67]:
verify.verify_dot_token()

mr. -> VBZ   |   mr -> NN
mr. -> FW   |   mr -> NN
dr. -> JJ   |   dr -> NN
a.m. -> VBD   |   am -> VBP
mr. -> VB   |   mr -> NN
a.m. -> RB   |   am -> VBP
u.n.r.r.a -> JJ   |   unrra -> NN
mrs. -> NN   |   mrs -> NN
p.m. -> NN   |   pm -> NN
prof. -> NN   |   prof -> NN
u.n.r.r.a -> RB   |   unrra -> NN
mr. -> JJ   |   mr -> NN
mr. -> VBD   |   mr -> NN
s.s.r -> NN   |   ssr -> NN
jr. -> NN   |   jr -> NN
dr. -> NN   |   dr -> NN
a.m. -> NN   |   am -> VBP
dr. -> VBZ   |   dr -> NN
mr. -> NNP   |   mr -> NN
st. -> JJ   |   st -> NN
mr. -> RBR   |   mr -> NN
maj. -> NN   |   maj -> NN
u.s.s.r. -> JJ   |   ussr -> NN
mr. -> NNS   |   mr -> NN
p.m. -> RB   |   pm -> NN
mr. -> NN   |   mr -> NN


##### apostrophe를 가진 token 처리

- apostrophe(')를 포함한 채로 tagging 하는 것이 더 정확하게 동작하는 것을 확인하였다.

In [68]:
verify.verify_apostrophe_token()

've -> VBP   |   ve -> NN
's -> POS   |   s -> NN
'll -> MD   |   ll -> NN


### adress POS

In [69]:
def dot_token_to_noun(data):
    result = []

    for tags in data:
        arr = []
        for tag in tags:
            if "." in tag[0] : arr.append((tag[0], "NN"))
            else : arr.append(tag)
        result.append(arr)
    
    return result

In [70]:
tagged_1_ = dot_token_to_noun(data=tagged_1)
tagged_2_ = dot_token_to_noun(data=tagged_2)
tagged_3_ = dot_token_to_noun(data=tagged_3)

## 4. Save PreProcessed Data

In [71]:
SAVE_ROOT = './Preprocessed/'

SAVE_1 = SAVE_ROOT + 'period-1/'
SAVE_2 = SAVE_ROOT + 'period-2/'
SAVE_3 = SAVE_ROOT + 'period-3/'

def to_pickle(data, file_name, root='./'):
    with open(root+file_name+'.pkl', 'wb') as f:
            pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

### Lemmatized data to pickle file

#### all pos

In [72]:
to_pickle(data=lemmatized_1, file_name="lemmatized-all", root=SAVE_1)
to_pickle(data=lemmatized_2, file_name="lemmatized-all", root=SAVE_2)
to_pickle(data=lemmatized_3, file_name="lemmatized-all", root=SAVE_3)

#### noun

In [73]:
to_pickle(data=extract_some_pos_ST(tagged_1_, tagList=tagList, pos_list=['noun']), \
    file_name="lemmatized-noun", root=SAVE_1)
to_pickle(data=extract_some_pos_ST(tagged_2_, tagList=tagList, pos_list=['noun']), \
    file_name="lemmatized-noun", root=SAVE_2)
to_pickle(data=extract_some_pos_ST(tagged_3_, tagList=tagList, pos_list=['noun']), \
    file_name="lemmatized-noun", root=SAVE_3)

#### verb

In [74]:
to_pickle(data=extract_some_pos_ST(tagged_1_, tagList=tagList, pos_list=['verb']), \
    file_name="lemmatized-verb", root=SAVE_1)
to_pickle(data=extract_some_pos_ST(tagged_2_, tagList=tagList, pos_list=['verb']), \
    file_name="lemmatized-verb", root=SAVE_2)
to_pickle(data=extract_some_pos_ST(tagged_3_, tagList=tagList, pos_list=['verb']), \
    file_name="lemmatized-verb", root=SAVE_3)

#### adjective

In [75]:
to_pickle(data=extract_some_pos_ST(tagged_1_, tagList=tagList, pos_list=['adjective']), \
    file_name="lemmatized-adjective", root=SAVE_1)
to_pickle(data=extract_some_pos_ST(tagged_2_, tagList=tagList, pos_list=['adjective']), \
    file_name="lemmatized-adjective", root=SAVE_2)
to_pickle(data=extract_some_pos_ST(tagged_3_, tagList=tagList, pos_list=['adjective']), \
    file_name="lemmatized-adjective", root=SAVE_3)

#### adverb

In [76]:
to_pickle(data=extract_some_pos_ST(tagged_1_, tagList=tagList, pos_list=['adverb']), \
    file_name="lemmatized-adverb", root=SAVE_1)
to_pickle(data=extract_some_pos_ST(tagged_2_, tagList=tagList, pos_list=['adverb']), \
    file_name="lemmatized-adverb", root=SAVE_2)
to_pickle(data=extract_some_pos_ST(tagged_3_, tagList=tagList, pos_list=['adverb']), \
    file_name="lemmatized-adverb", root=SAVE_3)

### Tagged data to pickle file

In [77]:
to_pickle(data=tagged_1_, file_name="tagged-all", root=SAVE_1)
to_pickle(data=tagged_2_, file_name="tagged-all", root=SAVE_2)
to_pickle(data=tagged_3_, file_name="tagged-all", root=SAVE_3)